In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
import re
import random
import string
from datetime import datetime
from progressbar import ProgressBar
import pandas as pd
import numpy as np
pbar = ProgressBar()
%config InlineBackend.figure_formats = ['retina']
import collections

from multiprocessing import Pool


In [2]:
# you can also import SoftwareEngine, HardwareType, SoftwareType, Popularity from random_user_agent.params
# you can also set number of user agents required by providing `limit` as parameter
software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   
user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)
# Get list of user agents.
user_agents = user_agent_rotator.get_user_agents()
user_agent = user_agent_rotator.get_random_user_agent()

In [3]:
# Get Random User Agent String.
def new_agent():
    return user_agent_rotator.get_random_user_agent()

In [4]:
df_user_18 = pd.read_csv('../project3/data/yelp_dataset_2019/user.csv', #skiprows = range(1, 1000000), nrows=1000000
                        )

/Users/chen/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
yelp_users = df_user_18['user_id'].unique()


In [8]:
yelp_users = list(yelp_users)

random.seed(4)
yelp_users = random.choices(yelp_users, k=100000)

yelp_users

['Hk_Np40f-7vGP4HRuVgUNA',
 'dMvs7AzNmnXweaZ6SJlNrw',
 'EU6l-a5rYymGq2jDFT863A',
 'rk6zgseTfZEpS8kuLe4q5Q',
 't57_frVTCf5e5J-xQxGV3g',
 's9W4BEiSIFmDPfaAugafJw',
 'iY4-hj_GUt0EIwK-Kds8Gw',
 '3CI1o3mjPPtDpo6jnolm3w',
 'oZ-hrmgL-E0ptfoLZu2hdw',
 '-oySYOATflw3CeCteZBFrQ',
 '9msXrvKZKK_hSz47LGfNOQ',
 'YK5SR_Llp7EFsO3r6w8coQ',
 'tnZP3PYL-hcqO97WKJ-nfw',
 '7mvZDwKuL_ZpXYypM-tAPw',
 'gfBWtb0s5TDT-Kq4s4QKyg',
 'oGcr0Ci5G6tEzV7GI3wNJQ',
 'Ho7LreAKzLwRu5_vcr4Zag',
 'ywA4YpvNdrT2mHUINvk8wA',
 'C2PYMthXAi1QzN3wOiBSKA',
 'rwrmbrvcr0Me-eJAvGSLLg',
 'B0yCM4Dgw5LFQYkSlBGmSg',
 'b5d0pWqyazsgu-LdSAfsrQ',
 'Cf6hJ9NRqOsJdEH_PUDN6w',
 'Zk9W444GkWVPbC_mjRWdlA',
 'zj6DxWysuiCPj2pJRoScFw',
 '0R4juIjFh3TFiWDCanGY9w',
 'Aq5CMs2YFJyWN-LY00I_Vg',
 'cXkdHRpRlXt2LwuYyLz0uw',
 'uuTy31TAW-olPMj3d4AGZA',
 'MUuydck5tjOqu8sjAQbN_g',
 'VyknzWSmgpglle3pyY6YfA',
 'kk3TA3BF5kpvoWtyRlVnmQ',
 'e_WsbBIe6dM5hJuVCJGAlg',
 'L-UpTe6VOVxmVym4nDHSCQ',
 'K-XhM4iQ9_iDI7a3PF60bA',
 'D_u_t1yXQL0RxTEmChZW1Q',
 'c91FOXToWWXdk1NPk0IfrA',
 

In [ ]:
r.choice(yelp_users)

In [ ]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
users_all = "https://www.yelp.com/"
#user_agent = {'User-agent': user_agent}
#driver = webdriver.Chrome(chromedriver)
#driver.get(user_all)
time.sleep(1)

In [ ]:
def scrape_yelp(user):
    #for user in user_list:
    try:
        #NEW PAGE
        user_agent=new_agent()
        link = 'https://www.yelp.com/user_details?userid='+str(user)
        page = requests.get(link, user_agent).text
        soup = BeautifulSoup(page, 'html5lib')

        #COLLECT VARIABLES
        if soup.find(class_ = 'badge-bar u-space-r1'):
            elite = soup.find(class_ = 'badge-bar u-space-r1').text
        else: 
            elite = None
            
        return user, elite
        #GO TO SLEEP
        time.sleep(.5*random.random())

        print(datetime.now(),user)

    except StopIteration:
        raise
    except Exception as e:
        print(e) # or whatever kind of logging you want
        pass


In [ ]:
def write_csv(records,start,end):
    file_name = str(start)+"_"+str(end)+".csv"
    pd.DataFrame(records).to_csv('../project3/data/yelp_dataset_2020/'+file_name, index=None, header=None)

In [ ]:
start = 52000
#for end in range(10000,len(yelp_users), 10000):
for end in range(53000,220001, 1000):
    print(datetime.now(), time.time, start, end)
    selected_yelp = yelp_users[start:end]
    p = Pool(10)  # Pool tells how many at a time
    records = p.map(scrape_yelp, selected_yelp)
    p.terminate()
    p.join()
    write_csv(records,start,end)
    records = ''
    start = end